# NYC Taxi Demand vs Used Car Price Analysis (Hybrid: Laptop/Cloud)
## Hypothesis: Taxi demand correlates with used car prices in New York areas

This notebook analyzes the correlation between NYC taxi trip density (2019-2020) and Craigslist used car prices in NY regions.

**Deployment Mode**: Toggle `LAPTOP_DEPLOYMENT` flag below to switch between:
- `True`: Pandas-only (Mac laptop friendly, < 16GB RAM recommended)
- `False`: Spark + Pandas (Cloudera AI / cluster deployment)

## 🔧 DEPLOYMENT CONFIGURATION

In [1]:
# ============================================================================
# TOGGLE THIS FLAG TO SWITCH DEPLOYMENT MODES
# ============================================================================
LAPTOP_DEPLOYMENT = True  # Set to False for Spark/Cloud deployment
# ============================================================================


In [ ]:

print(f"\n{'='*60}")
print(f"  DEPLOYMENT MODE: {'LAPTOP (Pandas)' if LAPTOP_DEPLOYMENT else 'CLOUD (Spark)'}")
print(f"{'='*60}\n")

if LAPTOP_DEPLOYMENT:
    print("✓ Running in Laptop mode (Pandas-only)")
    print("  - No Spark required")
    print("  - Lighter memory footprint")
    print("  - Good for datasets < 10GB")
else:
    print("✓ Running in Cloud mode (Spark + Pandas)")
    print("  - Distributed processing enabled")
    print("  - Scalable for large datasets")
    print("  - Optimized for Cloudera AI")

## 1. Setup and Package Installation

**💡 Recommended Alternative:** Use `uv sync` before opening the notebook for faster, reproducible installs!

```bash
# Laptop mode: uv sync --extra laptop --extra dev
# Cloud mode:  uv sync --extra cloud --extra dev
# See UV_SETUP.md for details
```

If you've already run `uv sync`, you can **skip this cell** entirely!

In [ ]:
# ============================================================================
# INSTALLATION METHOD: Choose pip or uv
# ============================================================================
USE_UV = False  # Set to True to use uv (faster), False for pip
# ============================================================================

import sys

if USE_UV:
    # Using uv (faster package installer)
    # Install uv first if not already installed: curl -LsSf https://astral.sh/uv/install.sh | sh
    print("📦 Installing packages with uv (fast mode)...")
    
    if LAPTOP_DEPLOYMENT:
        # Laptop mode packages
        !uv pip install --system kaggle pandas numpy matplotlib seaborn plotly pyarrow fastparquet
    else:
        # Cloud mode packages
        !uv pip install --system kaggle opendatasets pandas numpy matplotlib seaborn plotly
else:
    # Using standard pip
    print("📦 Installing packages with pip...")
    
    if LAPTOP_DEPLOYMENT:
        # Laptop mode packages
        !pip install kaggle pandas numpy matplotlib seaborn plotly pyarrow fastparquet -q
    else:
        # Cloud mode packages
        !pip install kaggle opendatasets pandas numpy matplotlib seaborn plotly -q

print("✓ Packages installed successfully!")

## 2. Import Libraries and Initialize

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

if LAPTOP_DEPLOYMENT:
    # Laptop mode: Pandas only
    print("Initializing Pandas environment...")
    # Optimize pandas for larger datasets
    pd.options.mode.chained_assignment = None
    print(f"✓ Pandas version: {pd.__version__}")
else:
    # Cloud mode: Import and initialize Spark
    print("Initializing Spark environment...")
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import *
    from pyspark.sql.types import *
    from pyspark.sql.window import Window
    
    spark = SparkSession.builder \
        .appName("TaxiCarPriceCorrelation") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    
    print(f"✓ Spark version: {spark.version}")
    print(f"✓ Pandas version: {pd.__version__}")

Initializing Pandas environment...
✓ Pandas version: 2.3.3


## 3. Download Datasets

**Note**: You need Kaggle API credentials:
1. Go to Kaggle → Account → Create New API Token
2. Place `kaggle.json` in `~/.kaggle/` directory
3. Run: `chmod 600 ~/.kaggle/kaggle.json`

In [3]:
# Create data directory
os.makedirs('data', exist_ok=True)

# Download Craigslist cars dataset
print("Downloading Craigslist cars dataset...")
!kaggle datasets download -d austinreese/craigslist-carstrucks-data -p ./data --unzip

# Download NYC taxi dataset
print("\nDownloading NYC Yellow Taxi dataset...")
!kaggle datasets download -d microize/newyork-yellow-taxi-trip-data-2020-2019 -p ./data --unzip

print("\n✓ Download complete!")
!ls -lh data/

Dataset URL: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data
License(s): CC0-1.0
 78%|███████████████████████████████         | 204M/262M [00:00<00:00, 2.14GB/s]
100%|████████████████████████████████████████| 262M/262M [00:00<00:00, 2.13GB/s]

Dataset URL: https://www.kaggle.com/datasets/microize/newyork-yellow-taxi-trip-data-2020-2019
License(s): ODbL-1.0
 99%|█████████████████████████████████████▌| 1.78G/1.81G [00:01<00:00, 2.01GB/s]
100%|██████████████████████████████████████| 1.81G/1.81G [00:01<00:00, 1.89GB/s]

✓ Download complete!
total 21200424
-rw-r--r--@ 1 eriksteinholz  staff    12K Feb  2 15:06 taxi+_zone_lookup.csv
drwxr-xr-x@ 9 eriksteinholz  staff   288B Feb  2 15:06 taxi_zones
-rw-r--r--@ 1 eriksteinholz  staff   1.3G Feb  2 15:03 vehicles.csv
-rw-r--r--@ 1 eriksteinholz  staff   655M Feb  2 15:06 yellow_tripdata_2019-01.csv
-rw-r--r--@ 1 eriksteinholz  staff   620M Feb  2 15:06 yellow_tripdata_2019-02.csv
-rw-r--r--@ 1 eriksteinholz  staff   693M 

## 4. Load Data

In [4]:
print("Loading Craigslist dataset...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Use Pandas with optimized dtypes
    craigslist_df = pd.read_csv(
        "data/vehicles.csv",
        low_memory=False
    )
    print(f"✓ Loaded {len(craigslist_df):,} records")
    print(f"\nColumns: {list(craigslist_df.columns)}")
    print(f"\nSample data:")
    print(craigslist_df.head())
else:
    # Cloud: Use Spark
    craigslist_df = spark.read.csv(
        "data/vehicles.csv",
        header=True,
        inferSchema=True
    )
    print("Craigslist Dataset Schema:")
    craigslist_df.printSchema()
    print(f"\n✓ Total records: {craigslist_df.count():,}")
    print("\nSample data:")
    craigslist_df.show(5, truncate=False)

Loading Craigslist dataset...
✓ Loaded 426,880 records

Columns: ['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color', 'image_url', 'description', 'county', 'state', 'lat', 'long', 'posting_date']

Sample data:
           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN  

In [5]:
# Check available states
print("States in Craigslist dataset:")

if LAPTOP_DEPLOYMENT:
    state_counts = craigslist_df['state'].value_counts().head(10)
    print(state_counts)
else:
    craigslist_df.groupBy("state").count().orderBy(desc("count")).show(10)

States in Craigslist dataset:
state
ca    50614
fl    28511
tx    22945
ny    19386
oh    17696
or    17104
mi    16900
nc    15277
wa    13861
pa    13753
Name: count, dtype: int64


In [7]:
print("\nLoading NYC Taxi dataset...")

# Detect file format
taxi_files = !ls data/*.parquet data/*.csv 2>/dev/null || echo "no_files"
is_parquet = any('.parquet' in f for f in taxi_files)

if LAPTOP_DEPLOYMENT:
    # Laptop: Use Pandas
    if is_parquet:
        import glob
        parquet_files = glob.glob("data/yellow_*.parquet") or glob.glob("data/*.parquet")
        if parquet_files:
            taxi_df = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)
        else:
            taxi_df = pd.read_parquet("data/")
    else:
        taxi_df = pd.read_csv("data/yellow_tripdata_2019-2020.csv", low_memory=False)
    
    print(f"✓ Loaded {len(taxi_df):,} records")
    print(f"\nColumns: {list(taxi_df.columns)}")
    print(f"\nSample data:")
    print(taxi_df.head())
else:
    # Cloud: Use Spark
    taxi_path = "data/yellow_*.parquet" if is_parquet else "data/*.csv"
    
    if is_parquet:
        taxi_df = spark.read.parquet(taxi_path)
    else:
        taxi_df = spark.read.csv(taxi_path, header=True, inferSchema=True)
    
    print("NYC Taxi Dataset Schema:")
    taxi_df.printSchema()
    print(f"\n✓ Total records: {taxi_df.count():,}")
    print("\nSample data:")
    taxi_df.show(5, truncate=False)


Loading NYC Taxi dataset...


ArrowKeyError: No type extension with name arrow.py_extension_type found

## 5. Data Preparation: Craigslist Cars

### 5.1 Filter to NY State and 2019-2020

In [ ]:
print("Filtering Craigslist data to NY state, 2019-2020...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas operations
    ny_cars_raw = craigslist_df[craigslist_df['state'] == 'ny'].copy()
    print(f"✓ NY car listings: {len(ny_cars_raw):,}")
    
    # Convert date column
    ny_cars_raw['posting_date'] = pd.to_datetime(ny_cars_raw['posting_date'], errors='coerce')
    
    # Show date range
    print(f"\nDate range: {ny_cars_raw['posting_date'].min()} to {ny_cars_raw['posting_date'].max()}")
    
else:
    # Cloud: Spark operations
    ny_cars_raw = craigslist_df.filter(col("state") == "ny")
    print(f"✓ NY car listings: {ny_cars_raw.count():,}")
    
    print("\nDate range in dataset:")
    ny_cars_raw.select(
        min("posting_date").alias("min_date"),
        max("posting_date").alias("max_date")
    ).show()

In [ ]:
print("Cleaning and filtering to 2019-2020...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas operations
    ny_cars_cleaned = ny_cars_raw[
        ny_cars_raw['price'].notna() &
        ny_cars_raw['year'].notna() &
        ny_cars_raw['posting_date'].notna()
    ].copy()
    
    # Filter to 2019-2020
    ny_cars_cleaned = ny_cars_cleaned[
        ny_cars_cleaned['posting_date'].dt.year.isin([2019, 2020])
    ]
    
    # Convert types
    ny_cars_cleaned['price'] = pd.to_numeric(ny_cars_cleaned['price'], errors='coerce')
    ny_cars_cleaned['year'] = pd.to_numeric(ny_cars_cleaned['year'], errors='coerce').astype('Int64')
    ny_cars_cleaned['odometer'] = pd.to_numeric(ny_cars_cleaned['odometer'], errors='coerce')
    
    print(f"✓ After filtering 2019-2020: {len(ny_cars_cleaned):,} records")
    
else:
    # Cloud: Spark operations
    ny_cars_cleaned = ny_cars_raw \
        .filter(col("price").isNotNull()) \
        .filter(col("year").isNotNull()) \
        .filter(col("posting_date").isNotNull()) \
        .withColumn("posting_date", to_timestamp(col("posting_date"))) \
        .filter(year(col("posting_date")).isin([2019, 2020])) \
        .withColumn("price", col("price").cast("double")) \
        .withColumn("year", col("year").cast("int")) \
        .withColumn("odometer", col("odometer").cast("double"))
    
    print(f"✓ After filtering 2019-2020: {ny_cars_cleaned.count():,} records")

### 5.2 Calculate Statistics and Remove Outliers

In [ ]:
print("Calculating price statistics...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas statistics
    price_stats = {
        'mean_price': ny_cars_cleaned['price'].mean(),
        'std_price': ny_cars_cleaned['price'].std(),
        'min_price': ny_cars_cleaned['price'].min(),
        'max_price': ny_cars_cleaned['price'].max(),
        'p01': ny_cars_cleaned['price'].quantile(0.01),
        'p99': ny_cars_cleaned['price'].quantile(0.99)
    }
else:
    # Cloud: Spark statistics
    price_stats = ny_cars_cleaned.select(
        mean("price").alias("mean_price"),
        stddev("price").alias("std_price"),
        expr("percentile_approx(price, 0.01)").alias("p01"),
        expr("percentile_approx(price, 0.99)").alias("p99"),
        min("price").alias("min_price"),
        max("price").alias("max_price")
    ).collect()[0].asDict()

print("\n=== PRICE STATISTICS ===")
print(f"Mean: ${price_stats['mean_price']:,.2f}")
print(f"Std Dev: ${price_stats['std_price']:,.2f}")
print(f"Min: ${price_stats['min_price']:,.2f}")
print(f"Max: ${price_stats['max_price']:,.2f}")
print(f"1st percentile: ${price_stats['p01']:,.2f}")
print(f"99th percentile: ${price_stats['p99']:,.2f}")

# Define outlier thresholds
price_lower = max(500, price_stats['p01'])
price_upper = min(75000, price_stats['p99'])

print(f"\n✓ Outlier thresholds: ${price_lower:,.2f} - ${price_upper:,.2f}")

In [ ]:
print("Removing outliers...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas filtering
    outliers_mask = (ny_cars_cleaned['price'] < price_lower) | (ny_cars_cleaned['price'] > price_upper)
    outliers_removed = ny_cars_cleaned[outliers_mask]
    ny_cars = ny_cars_cleaned[~outliers_mask].copy()
    
    print(f"\n=== OUTLIER REMOVAL REPORT ===")
    print(f"Records before: {len(ny_cars_cleaned):,}")
    print(f"Outliers removed: {len(outliers_removed):,} ({len(outliers_removed)/len(ny_cars_cleaned)*100:.2f}%)")
    print(f"Records after: {len(ny_cars):,}")
    
    print("\nOutlier breakdown:")
    too_low = (outliers_removed['price'] < price_lower).sum()
    too_high = (outliers_removed['price'] > price_upper).sum()
    print(f"  Too low (< ${price_lower:,.0f}): {too_low:,}")
    print(f"  Too high (> ${price_upper:,.0f}): {too_high:,}")
    
else:
    # Cloud: Spark filtering
    outliers_removed = ny_cars_cleaned.filter(
        (col("price") < price_lower) | (col("price") > price_upper)
    )
    ny_cars = ny_cars_cleaned.filter(
        (col("price") >= price_lower) & (col("price") <= price_upper)
    )
    
    print(f"\n=== OUTLIER REMOVAL REPORT ===")
    print(f"Records before: {ny_cars_cleaned.count():,}")
    print(f"Outliers removed: {outliers_removed.count():,} ({outliers_removed.count()/ny_cars_cleaned.count()*100:.2f}%)")
    print(f"Records after: {ny_cars.count():,}")
    
    print("\nOutlier breakdown:")
    outliers_removed.select(
        count(when(col("price") < price_lower, 1)).alias("too_low"),
        count(when(col("price") > price_upper, 1)).alias("too_high")
    ).show()

### 5.3 Add Derived Columns

In [ ]:
print("Adding derived columns for dimensional analysis...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas operations
    ny_cars_final = ny_cars.copy()
    
    # Date dimensions
    ny_cars_final['date'] = ny_cars_final['posting_date'].dt.date
    ny_cars_final['year_month'] = ny_cars_final['posting_date'].dt.strftime('%Y-%m')
    ny_cars_final['year_week'] = ny_cars_final['posting_date'].dt.strftime('%Y-%U')
    
    # Vehicle age
    ny_cars_final['vehicle_age'] = ny_cars_final['posting_date'].dt.year - ny_cars_final['year']
    ny_cars_final['age_category'] = pd.cut(
        ny_cars_final['vehicle_age'],
        bins=[-np.inf, 3, 7, 12, np.inf],
        labels=['0-3 years', '4-7 years', '8-12 years', '12+ years']
    )
    
    # Clean text fields
    ny_cars_final['vehicle_type_clean'] = ny_cars_final['type'].fillna('unknown').str.lower().str.strip()
    ny_cars_final['manufacturer_clean'] = ny_cars_final['manufacturer'].fillna('unknown').str.lower().str.strip()
    ny_cars_final['region_clean'] = ny_cars_final['region'].fillna('unknown').str.lower().str.strip()
    
    print(f"✓ Final car dataset: {len(ny_cars_final):,} records")
    print("\nSample:")
    print(ny_cars_final[[
        'date', 'year_month', 'price', 'manufacturer_clean',
        'vehicle_type_clean', 'age_category', 'region_clean'
    ]].head(10))
    
else:
    # Cloud: Spark operations
    ny_cars_final = ny_cars \
        .withColumn("date", to_date(col("posting_date"))) \
        .withColumn("year_month", date_format(col("posting_date"), "yyyy-MM")) \
        .withColumn("year_week", date_format(col("posting_date"), "yyyy-ww")) \
        .withColumn("vehicle_age", year(col("posting_date")) - col("year")) \
        .withColumn(
            "age_category",
            when(col("vehicle_age") <= 3, "0-3 years")
            .when(col("vehicle_age") <= 7, "4-7 years")
            .when(col("vehicle_age") <= 12, "8-12 years")
            .otherwise("12+ years")
        ) \
        .withColumn(
            "vehicle_type_clean",
            coalesce(lower(trim(col("type"))), lit("unknown"))
        ) \
        .withColumn(
            "manufacturer_clean",
            coalesce(lower(trim(col("manufacturer"))), lit("unknown"))
        ) \
        .withColumn(
            "region_clean",
            coalesce(lower(trim(col("region"))), lit("unknown"))
        )
    
    print("✓ Final car dataset prepared")
    ny_cars_final.select(
        "date", "year_month", "price", "manufacturer_clean",
        "vehicle_type_clean", "age_category", "region_clean"
    ).show(10)

## 6. Data Preparation: NYC Taxi

### 6.1 Identify Columns and Filter 2019-2020

In [ ]:
print("Identifying datetime columns...")

if LAPTOP_DEPLOYMENT:
    taxi_columns = list(taxi_df.columns)
else:
    taxi_columns = taxi_df.columns

pickup_col = [c for c in taxi_columns if 'pickup' in c.lower() and 'datetime' in c.lower()][0]
dropoff_col = [c for c in taxi_columns if 'dropoff' in c.lower() and 'datetime' in c.lower()][0]

print(f"✓ Using columns: {pickup_col}, {dropoff_col}")

In [ ]:
print("Cleaning and filtering taxi data to 2019-2020...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas operations
    taxi_cleaned = taxi_df[taxi_df[pickup_col].notna()].copy()
    
    # Convert datetime
    taxi_cleaned['pickup_datetime'] = pd.to_datetime(taxi_cleaned[pickup_col], errors='coerce')
    taxi_cleaned['dropoff_datetime'] = pd.to_datetime(taxi_cleaned[dropoff_col], errors='coerce')
    
    # Filter to 2019-2020
    taxi_cleaned = taxi_cleaned[
        taxi_cleaned['pickup_datetime'].dt.year.isin([2019, 2020]) &
        (taxi_cleaned['pickup_datetime'] < taxi_cleaned['dropoff_datetime'])
    ]
    
    print(f"✓ Taxi records in 2019-2020: {len(taxi_cleaned):,}")
    
else:
    # Cloud: Spark operations
    taxi_cleaned = taxi_df \
        .filter(col(pickup_col).isNotNull()) \
        .withColumn("pickup_datetime", to_timestamp(col(pickup_col))) \
        .withColumn("dropoff_datetime", to_timestamp(col(dropoff_col))) \
        .filter(year(col("pickup_datetime")).isin([2019, 2020])) \
        .filter(col("pickup_datetime") < col("dropoff_datetime"))
    
    print(f"✓ Taxi records in 2019-2020: {taxi_cleaned.count():,}")

### 6.2 Remove Distance Outliers

In [ ]:
# Check for distance column
distance_col = None
for col_name in ['trip_distance', 'distance']:
    if col_name in taxi_columns:
        distance_col = col_name
        break

if distance_col:
    print(f"Removing trip distance outliers (using column: {distance_col})...")
    
    if LAPTOP_DEPLOYMENT:
        # Laptop: Pandas operations
        p99_distance = taxi_cleaned[distance_col].quantile(0.99)
        max_trip_distance = min(50, p99_distance)
        
        outliers_count = (
            (taxi_cleaned[distance_col] <= 0) | 
            (taxi_cleaned[distance_col] > max_trip_distance)
        ).sum()
        
        taxi_cleaned = taxi_cleaned[
            (taxi_cleaned[distance_col] > 0) & 
            (taxi_cleaned[distance_col] <= max_trip_distance)
        ]
        
        print(f"\n=== TAXI OUTLIER REMOVAL ===")
        print(f"Outliers removed: {outliers_count:,}")
        print(f"✓ Remaining records: {len(taxi_cleaned):,}")
        
    else:
        # Cloud: Spark operations
        taxi_distance_stats = taxi_cleaned.select(
            expr(f"percentile_approx({distance_col}, 0.99)").alias("p99_distance"),
            max(distance_col).alias("max_distance")
        ).collect()[0]
        
        max_trip_distance = min(50, taxi_distance_stats['p99_distance'])
        
        outliers_taxi = taxi_cleaned.filter(
            (col(distance_col) <= 0) | (col(distance_col) > max_trip_distance)
        )
        
        taxi_cleaned = taxi_cleaned.filter(
            (col(distance_col) > 0) & (col(distance_col) <= max_trip_distance)
        )
        
        print(f"\n=== TAXI OUTLIER REMOVAL ===")
        print(f"Outliers removed: {outliers_taxi.count():,}")
        print(f"✓ Remaining records: {taxi_cleaned.count():,}")
else:
    print("⚠ No distance column found, skipping distance outlier removal")

### 6.3 Add Time Dimensions

In [ ]:
print("Adding time dimensions to taxi data...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas operations
    taxi_final = taxi_cleaned.copy()
    
    taxi_final['pickup_date'] = taxi_final['pickup_datetime'].dt.date
    taxi_final['pickup_year_month'] = taxi_final['pickup_datetime'].dt.strftime('%Y-%m')
    taxi_final['pickup_year_week'] = taxi_final['pickup_datetime'].dt.strftime('%Y-%U')
    taxi_final['pickup_hour'] = taxi_final['pickup_datetime'].dt.hour
    taxi_final['dropoff_date'] = taxi_final['dropoff_datetime'].dt.date
    taxi_final['trip_duration_min'] = (
        (taxi_final['dropoff_datetime'] - taxi_final['pickup_datetime']).dt.total_seconds() / 60
    )
    
    print(f"✓ Taxi dataset prepared: {len(taxi_final):,} records")
    print("\nSample:")
    print(taxi_final[['pickup_date', 'pickup_year_month', 'pickup_hour', 'trip_duration_min']].head(10))
    
else:
    # Cloud: Spark operations
    taxi_final = taxi_cleaned \
        .withColumn("pickup_date", to_date(col("pickup_datetime"))) \
        .withColumn("pickup_year_month", date_format(col("pickup_datetime"), "yyyy-MM")) \
        .withColumn("pickup_year_week", date_format(col("pickup_datetime"), "yyyy-ww")) \
        .withColumn("pickup_hour", hour(col("pickup_datetime"))) \
        .withColumn("dropoff_date", to_date(col("dropoff_datetime"))) \
        .withColumn("trip_duration_min",
                    (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) / 60)
    
    print("✓ Taxi dataset prepared")
    taxi_final.select(
        "pickup_date", "pickup_year_month", "pickup_hour", "trip_duration_min"
    ).show(10)

## 7. Build Dimensional Model - Taxi Demand Facts

In [ ]:
print("Creating taxi demand aggregations...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas aggregations
    
    # Daily pickup density
    pickup_density_daily = taxi_final.groupby('pickup_date').agg({
        'pickup_datetime': 'count',
        'trip_duration_min': 'mean'
    }).rename(columns={
        'pickup_datetime': 'pickup_trip_count',
        'trip_duration_min': 'avg_trip_duration'
    }).reset_index().rename(columns={'pickup_date': 'date'})
    
    # Daily dropoff density
    dropoff_density_daily = taxi_final.groupby('dropoff_date').agg({
        'dropoff_datetime': 'count'
    }).rename(columns={
        'dropoff_datetime': 'dropoff_trip_count'
    }).reset_index().rename(columns={'dropoff_date': 'date'})
    
    # Merge
    taxi_demand_daily = pickup_density_daily.merge(
        dropoff_density_daily, on='date', how='outer'
    ).fillna(0)
    
    taxi_demand_daily['total_trip_count'] = (
        taxi_demand_daily['pickup_trip_count'] + taxi_demand_daily['dropoff_trip_count']
    )
    taxi_demand_daily['date'] = pd.to_datetime(taxi_demand_daily['date'])
    taxi_demand_daily['year_month'] = taxi_demand_daily['date'].dt.strftime('%Y-%m')
    taxi_demand_daily['year_week'] = taxi_demand_daily['date'].dt.strftime('%Y-%U')
    
    print(f"✓ Daily taxi demand records: {len(taxi_demand_daily):,}")
    print("\nSample:")
    print(taxi_demand_daily.head(10))
    
else:
    # Cloud: Spark aggregations
    pickup_density_daily = taxi_final \
        .groupBy("pickup_date") \
        .agg(
            count("*").alias("pickup_trip_count"),
            avg("trip_duration_min").alias("avg_trip_duration")
        ) \
        .withColumn("trip_type", lit("pickup"))
    
    dropoff_density_daily = taxi_final \
        .groupBy("dropoff_date") \
        .agg(
            count("*").alias("dropoff_trip_count"),
        ) \
        .withColumnRenamed("dropoff_date", "date")
    
    taxi_demand_daily = pickup_density_daily \
        .withColumnRenamed("pickup_date", "date") \
        .join(dropoff_density_daily, "date", "outer") \
        .fillna(0) \
        .withColumn("total_trip_count", col("pickup_trip_count") + col("dropoff_trip_count")) \
        .withColumn("year_month", date_format(col("date"), "yyyy-MM")) \
        .withColumn("year_week", date_format(col("date"), "yyyy-ww"))
    
    print(f"✓ Daily taxi demand records: {taxi_demand_daily.count():,}")
    taxi_demand_daily.orderBy("date").show(10)

In [ ]:
# Weekly aggregation
print("Creating weekly aggregation...")

if LAPTOP_DEPLOYMENT:
    taxi_demand_weekly = taxi_demand_daily.groupby('year_week').agg({
        'pickup_trip_count': 'sum',
        'dropoff_trip_count': 'sum',
        'total_trip_count': 'sum',
        'avg_trip_duration': 'mean'
    }).reset_index()
    
    print(f"✓ Weekly taxi demand records: {len(taxi_demand_weekly):,}")
else:
    taxi_demand_weekly = taxi_demand_daily \
        .groupBy("year_week") \
        .agg(
            sum("pickup_trip_count").alias("pickup_trip_count"),
            sum("dropoff_trip_count").alias("dropoff_trip_count"),
            sum("total_trip_count").alias("total_trip_count"),
            avg("avg_trip_duration").alias("avg_trip_duration")
        )
    
    print(f"✓ Weekly taxi demand records: {taxi_demand_weekly.count():,}")

In [ ]:
# Monthly aggregation
print("Creating monthly aggregation...")

if LAPTOP_DEPLOYMENT:
    taxi_demand_monthly = taxi_demand_daily.groupby('year_month').agg({
        'pickup_trip_count': 'sum',
        'dropoff_trip_count': 'sum',
        'total_trip_count': 'sum',
        'avg_trip_duration': 'mean',
        'date': 'count'
    }).rename(columns={'date': 'days_in_period'}).reset_index()
    
    taxi_demand_monthly['avg_daily_trips'] = (
        taxi_demand_monthly['total_trip_count'] / taxi_demand_monthly['days_in_period']
    )
    
    print(f"✓ Monthly taxi demand records: {len(taxi_demand_monthly):,}")
    print("\nSample:")
    print(taxi_demand_monthly)
else:
    taxi_demand_monthly = taxi_demand_daily \
        .groupBy("year_month") \
        .agg(
            sum("pickup_trip_count").alias("pickup_trip_count"),
            sum("dropoff_trip_count").alias("dropoff_trip_count"),
            sum("total_trip_count").alias("total_trip_count"),
            avg("avg_trip_duration").alias("avg_trip_duration"),
            count("date").alias("days_in_period")
        ) \
        .withColumn("avg_daily_trips", col("total_trip_count") / col("days_in_period"))
    
    print(f"✓ Monthly taxi demand records: {taxi_demand_monthly.count():,}")
    taxi_demand_monthly.orderBy("year_month").show()

## 8. Build Dimensional Model - Car Price Facts

In [ ]:
print("Creating car price aggregations...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas aggregations
    
    # Daily
    car_price_daily = ny_cars_final.groupby('date').agg({
        'price': ['median', 'mean', 'count', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]
    })
    car_price_daily.columns = ['median_price', 'avg_price', 'listing_count', 'p25_price', 'p75_price']
    car_price_daily = car_price_daily.reset_index()
    car_price_daily['date'] = pd.to_datetime(car_price_daily['date'])
    car_price_daily['year_month'] = car_price_daily['date'].dt.strftime('%Y-%m')
    car_price_daily['year_week'] = car_price_daily['date'].dt.strftime('%Y-%U')
    
    print(f"✓ Daily car price records: {len(car_price_daily):,}")
    
    # Weekly
    car_price_weekly = ny_cars_final.groupby('year_week').agg({
        'price': ['median', 'mean', 'count']
    })
    car_price_weekly.columns = ['median_price', 'avg_price', 'listing_count']
    car_price_weekly = car_price_weekly.reset_index()
    
    print(f"✓ Weekly car price records: {len(car_price_weekly):,}")
    
    # Monthly
    car_price_monthly = ny_cars_final.groupby('year_month').agg({
        'price': ['median', 'mean', 'count', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]
    })
    car_price_monthly.columns = ['median_price', 'avg_price', 'listing_count', 'p25_price', 'p75_price']
    car_price_monthly = car_price_monthly.reset_index()
    
    print(f"✓ Monthly car price records: {len(car_price_monthly):,}")
    print("\nSample:")
    print(car_price_monthly)
    
else:
    # Cloud: Spark aggregations
    
    # Daily
    car_price_daily = ny_cars_final \
        .groupBy("date") \
        .agg(
            expr("percentile_approx(price, 0.5)").alias("median_price"),
            avg("price").alias("avg_price"),
            count("*").alias("listing_count"),
            expr("percentile_approx(price, 0.25)").alias("p25_price"),
            expr("percentile_approx(price, 0.75)").alias("p75_price")
        ) \
        .withColumn("year_month", date_format(col("date"), "yyyy-MM")) \
        .withColumn("year_week", date_format(col("date"), "yyyy-ww"))
    
    print(f"✓ Daily car price records: {car_price_daily.count():,}")
    
    # Weekly
    car_price_weekly = ny_cars_final \
        .groupBy("year_week") \
        .agg(
            expr("percentile_approx(price, 0.5)").alias("median_price"),
            avg("price").alias("avg_price"),
            count("*").alias("listing_count")
        )
    
    print(f"✓ Weekly car price records: {car_price_weekly.count():,}")
    
    # Monthly
    car_price_monthly = ny_cars_final \
        .groupBy("year_month") \
        .agg(
            expr("percentile_approx(price, 0.5)").alias("median_price"),
            avg("price").alias("avg_price"),
            count("*").alias("listing_count"),
            expr("percentile_approx(price, 0.25)").alias("p25_price"),
            expr("percentile_approx(price, 0.75)").alias("p75_price")
        )
    
    print(f"✓ Monthly car price records: {car_price_monthly.count():,}")
    car_price_monthly.orderBy("year_month").show()

In [ ]:
# Car price by vehicle type (monthly)
print("Creating price by vehicle type...")

if LAPTOP_DEPLOYMENT:
    car_price_by_type_monthly = ny_cars_final.groupby(['year_month', 'vehicle_type_clean']).agg({
        'price': ['median', 'count']
    })
    car_price_by_type_monthly.columns = ['median_price', 'listing_count']
    car_price_by_type_monthly = car_price_by_type_monthly.reset_index()
    car_price_by_type_monthly = car_price_by_type_monthly[
        car_price_by_type_monthly['listing_count'] >= 10
    ]
    
    print(f"✓ Car price by type (monthly): {len(car_price_by_type_monthly):,}")
else:
    car_price_by_type_monthly = ny_cars_final \
        .groupBy("year_month", "vehicle_type_clean") \
        .agg(
            expr("percentile_approx(price, 0.5)").alias("median_price"),
            count("*").alias("listing_count")
        ) \
        .filter(col("listing_count") >= 10)
    
    print(f"✓ Car price by type (monthly): {car_price_by_type_monthly.count():,}")

In [ ]:
# Car price by age category (monthly)
print("Creating price by age category...")

if LAPTOP_DEPLOYMENT:
    car_price_by_age_monthly = ny_cars_final.groupby(['year_month', 'age_category']).agg({
        'price': ['median', 'count']
    })
    car_price_by_age_monthly.columns = ['median_price', 'listing_count']
    car_price_by_age_monthly = car_price_by_age_monthly.reset_index()
    car_price_by_age_monthly = car_price_by_age_monthly[
        car_price_by_age_monthly['listing_count'] >= 10
    ]
    
    print(f"✓ Car price by age (monthly): {len(car_price_by_age_monthly):,}")
else:
    car_price_by_age_monthly = ny_cars_final \
        .groupBy("year_month", "age_category") \
        .agg(
            expr("percentile_approx(price, 0.5)").alias("median_price"),
            count("*").alias("listing_count")
        ) \
        .filter(col("listing_count") >= 10)
    
    print(f"✓ Car price by age (monthly): {car_price_by_age_monthly.count():,}")

## 9. Combine Fact Tables (Join)

In [ ]:
print("Joining taxi demand and car price facts...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Pandas merge
    
    # Daily
    combined_daily = taxi_demand_daily.merge(
        car_price_daily[['date', 'median_price', 'avg_price', 'listing_count', 'p25_price', 'p75_price']],
        on='date',
        how='inner'
    ).sort_values('date')
    
    print(f"✓ Combined daily records: {len(combined_daily):,}")
    
    # Weekly
    combined_weekly = taxi_demand_weekly.merge(
        car_price_weekly,
        on='year_week',
        how='inner'
    ).sort_values('year_week')
    
    print(f"✓ Combined weekly records: {len(combined_weekly):,}")
    
    # Monthly
    combined_monthly = taxi_demand_monthly.merge(
        car_price_monthly,
        on='year_month',
        how='inner'
    )
    combined_monthly = combined_monthly.rename(columns={
        'pickup_trip_count': 'monthly_pickups',
        'dropoff_trip_count': 'monthly_dropoffs',
        'total_trip_count': 'monthly_total_trips'
    }).sort_values('year_month')
    
    print(f"✓ Combined monthly records: {len(combined_monthly):,}")
    print("\nMonthly combined data:")
    print(combined_monthly)
    
else:
    # Cloud: Spark join
    
    # Daily
    combined_daily = taxi_demand_daily \
        .join(car_price_daily, "date", "inner") \
        .select(
            "date",
            "year_month",
            "year_week",
            "pickup_trip_count",
            "dropoff_trip_count",
            "total_trip_count",
            "avg_trip_duration",
            "median_price",
            "avg_price",
            "listing_count",
            "p25_price",
            "p75_price"
        ) \
        .orderBy("date")
    
    print(f"✓ Combined daily records: {combined_daily.count():,}")
    
    # Weekly
    combined_weekly = taxi_demand_weekly \
        .join(car_price_weekly, "year_week", "inner") \
        .orderBy("year_week")
    
    print(f"✓ Combined weekly records: {combined_weekly.count():,}")
    
    # Monthly
    combined_monthly = taxi_demand_monthly \
        .join(car_price_monthly, "year_month", "inner") \
        .select(
            "year_month",
            col("pickup_trip_count").alias("monthly_pickups"),
            col("dropoff_trip_count").alias("monthly_dropoffs"),
            col("total_trip_count").alias("monthly_total_trips"),
            "avg_daily_trips",
            "avg_trip_duration",
            "median_price",
            "avg_price",
            "listing_count",
            "p25_price",
            "p75_price"
        ) \
        .orderBy("year_month")
    
    print(f"✓ Combined monthly records: {combined_monthly.count():,}")
    combined_monthly.show()

## 10. Convert to Pandas for Analysis & Visualization

In [ ]:
print("Converting to Pandas for correlation analysis and visualization...")

if LAPTOP_DEPLOYMENT:
    # Already in Pandas, just assign
    combined_daily_pd = combined_daily
    combined_weekly_pd = combined_weekly
    combined_monthly_pd = combined_monthly
    price_by_type_pd = car_price_by_type_monthly
    price_by_age_pd = car_price_by_age_monthly
else:
    # Convert from Spark to Pandas
    combined_daily_pd = combined_daily.toPandas()
    combined_weekly_pd = combined_weekly.toPandas()
    combined_monthly_pd = combined_monthly.toPandas()
    price_by_type_pd = car_price_by_type_monthly.toPandas()
    price_by_age_pd = car_price_by_age_monthly.toPandas()

print("✓ Data ready for analysis")

## 11. Correlation Analysis

In [ ]:
print("=" * 60)
print("CORRELATION ANALYSIS")
print("=" * 60)

print("\n1. Daily Granularity:")
print(f"   Pickup trips vs Median price: {combined_daily_pd['pickup_trip_count'].corr(combined_daily_pd['median_price']):.4f}")
print(f"   Dropoff trips vs Median price: {combined_daily_pd['dropoff_trip_count'].corr(combined_daily_pd['median_price']):.4f}")
print(f"   Total trips vs Median price: {combined_daily_pd['total_trip_count'].corr(combined_daily_pd['median_price']):.4f}")

print("\n2. Weekly Granularity:")
print(f"   Pickup trips vs Median price: {combined_weekly_pd['pickup_trip_count'].corr(combined_weekly_pd['median_price']):.4f}")
print(f"   Dropoff trips vs Median price: {combined_weekly_pd['dropoff_trip_count'].corr(combined_weekly_pd['median_price']):.4f}")
print(f"   Total trips vs Median price: {combined_weekly_pd['total_trip_count'].corr(combined_weekly_pd['median_price']):.4f}")

print("\n3. Monthly Granularity:")
print(f"   Monthly pickups vs Median price: {combined_monthly_pd['monthly_pickups'].corr(combined_monthly_pd['median_price']):.4f}")
print(f"   Monthly dropoffs vs Median price: {combined_monthly_pd['monthly_dropoffs'].corr(combined_monthly_pd['median_price']):.4f}")
print(f"   Monthly total trips vs Median price: {combined_monthly_pd['monthly_total_trips'].corr(combined_monthly_pd['median_price']):.4f}")
print(f"   Avg daily trips vs Median price: {combined_monthly_pd['avg_daily_trips'].corr(combined_monthly_pd['median_price']):.4f}")

In [ ]:
# Lagged correlation analysis
combined_monthly_pd_sorted = combined_monthly_pd.sort_values('year_month').copy()

print("\n=== LAGGED CORRELATION ANALYSIS ===")
print("(Does taxi demand predict future car prices?)\n")

for lag in [1, 2, 3]:
    combined_monthly_pd_sorted[f'median_price_lag_{lag}'] = combined_monthly_pd_sorted['median_price'].shift(lag)
    corr = combined_monthly_pd_sorted['monthly_total_trips'].corr(
        combined_monthly_pd_sorted[f'median_price_lag_{lag}']
    )
    print(f"Lag {lag} month: Trips vs Price: {corr:.4f}")

print("\n(Does car price predict future taxi demand?)\n")
for lag in [1, 2, 3]:
    combined_monthly_pd_sorted[f'trips_lag_{lag}'] = combined_monthly_pd_sorted['monthly_total_trips'].shift(lag)
    corr = combined_monthly_pd_sorted['median_price'].corr(
        combined_monthly_pd_sorted[f'trips_lag_{lag}']
    )
    print(f"Lag {lag} month: Price vs Trips: {corr:.4f}")

## 12. Summary Statistics

In [ ]:
print("\n" + "=" * 60)
print("DATASET SUMMARY")
print("=" * 60)

print(f"\nTime period: 2019-2020")

if LAPTOP_DEPLOYMENT:
    print(f"Taxi trips analyzed: {len(taxi_final):,}")
    print(f"NY car listings analyzed: {len(ny_cars_final):,}")
else:
    print(f"Taxi trips analyzed: {taxi_final.count():,}")
    print(f"NY car listings analyzed: {ny_cars_final.count():,}")

print(f"\nDaily observations: {len(combined_daily_pd):,}")
print(f"Weekly observations: {len(combined_weekly_pd):,}")
print(f"Monthly observations: {len(combined_monthly_pd):,}")

print("\n=== KEY INSIGHTS ===")
print(f"Average daily taxi trips: {combined_monthly_pd['avg_daily_trips'].mean():,.0f}")
print(f"Average median car price: ${combined_monthly_pd['median_price'].mean():,.2f}")
print(f"Price range: ${combined_monthly_pd['median_price'].min():,.2f} - ${combined_monthly_pd['median_price'].max():,.2f}")
print(f"Trip volume range: {combined_monthly_pd['monthly_total_trips'].min():,.0f} - {combined_monthly_pd['monthly_total_trips'].max():,.0f}")

## 13. Export Combined Datasets

In [ ]:
print("Saving combined datasets...")

if LAPTOP_DEPLOYMENT:
    # Laptop: Save as parquet using Pandas
    combined_daily_pd.to_parquet('data/combined_daily.parquet', index=False)
    combined_weekly_pd.to_parquet('data/combined_weekly.parquet', index=False)
    combined_monthly_pd.to_parquet('data/combined_monthly.parquet', index=False)
    price_by_type_pd.to_parquet('data/price_by_type_monthly.parquet', index=False)
    price_by_age_pd.to_parquet('data/price_by_age_monthly.parquet', index=False)
else:
    # Cloud: Save using Spark
    combined_daily.write.mode('overwrite').parquet('data/combined_daily.parquet')
    combined_weekly.write.mode('overwrite').parquet('data/combined_weekly.parquet')
    combined_monthly.write.mode('overwrite').parquet('data/combined_monthly.parquet')
    car_price_by_type_monthly.write.mode('overwrite').parquet('data/price_by_type_monthly.parquet')
    car_price_by_age_monthly.write.mode('overwrite').parquet('data/price_by_age_monthly.parquet')

print("✓ Combined datasets saved to parquet files")

## 14. Visualizations

The visualization code below works the same regardless of deployment mode (since we've converted to Pandas).

### Graph 1: Time Series Overlay

In [ ]:
# Prepare data
combined_monthly_pd['year_month_dt'] = pd.to_datetime(combined_monthly_pd['year_month'])

# Create figure with secondary y-axis
fig = go.Figure()

# Add taxi trips trace
fig.add_trace(go.Scatter(
    x=combined_monthly_pd['year_month_dt'],
    y=combined_monthly_pd['monthly_total_trips'],
    name='Total Taxi Trips',
    line=dict(color='blue', width=2),
    yaxis='y'
))

# Add car price trace
fig.add_trace(go.Scatter(
    x=combined_monthly_pd['year_month_dt'],
    y=combined_monthly_pd['median_price'],
    name='Median Car Price',
    line=dict(color='red', width=2),
    yaxis='y2'
))

# Update layout
fig.update_layout(
    title='NYC Taxi Demand vs Used Car Prices Over Time (2019-2020)',
    xaxis=dict(title='Month'),
    yaxis=dict(
        title='Total Taxi Trips',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title='Median Car Price ($)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red'),
        overlaying='y',
        side='right'
    ),
    hovermode='x unified',
    height=500
)

fig.show()

print("\nGraph 1: If hypothesis is true, you should see the two lines moving in similar patterns.")

### Graph 2: Scatter Plot with Trend Line

In [ ]:
# Create scatter plot
fig = px.scatter(
    combined_monthly_pd,
    x='monthly_total_trips',
    y='median_price',
    color='year_month_dt',
    trendline='ols',
    labels={
        'monthly_total_trips': 'Total Monthly Taxi Trips',
        'median_price': 'Median Car Price ($)',
        'year_month_dt': 'Month'
    },
    title='Correlation: Taxi Trip Volume vs Car Prices',
    hover_data=['year_month']
)

fig.update_layout(height=500)
fig.show()

corr_val = combined_monthly_pd['monthly_total_trips'].corr(combined_monthly_pd['median_price'])
print(f"\nGraph 2: Correlation coefficient = {corr_val:.4f}")
print("If hypothesis is true, expect positive correlation (upward trend line).")

### Graph 3: Correlation Heatmap

In [ ]:
# Select numeric columns for correlation
corr_cols = [
    'monthly_pickups', 'monthly_dropoffs', 'monthly_total_trips',
    'avg_daily_trips', 'avg_trip_duration', 'median_price', 'avg_price'
]

corr_matrix = combined_monthly_pd[corr_cols].corr()

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values,
    texttemplate='%{text:.3f}',
    textfont={"size": 10}
))

fig.update_layout(
    title='Correlation Matrix: All Variables',
    height=600,
    xaxis={'side': 'bottom'}
)

fig.show()

print("\nGraph 3: Look for strong correlations (close to +1 or -1) between trip metrics and price.")

### Graph 4: Lagged Correlation Plot

In [ ]:
# Calculate lagged correlations
lags = list(range(0, 6))
trips_to_price_corr = []
price_to_trips_corr = []

combined_monthly_sorted = combined_monthly_pd.sort_values('year_month')

for lag in lags:
    if lag == 0:
        trips_to_price_corr.append(
            combined_monthly_sorted['monthly_total_trips'].corr(combined_monthly_sorted['median_price'])
        )
        price_to_trips_corr.append(
            combined_monthly_sorted['median_price'].corr(combined_monthly_sorted['monthly_total_trips'])
        )
    else:
        trips_to_price_corr.append(
            combined_monthly_sorted['monthly_total_trips'].iloc[:-lag].corr(
                combined_monthly_sorted['median_price'].iloc[lag:]
            )
        )
        price_to_trips_corr.append(
            combined_monthly_sorted['median_price'].iloc[:-lag].corr(
                combined_monthly_sorted['monthly_total_trips'].iloc[lag:]
            )
        )

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=lags,
    y=trips_to_price_corr,
    mode='lines+markers',
    name='Trips → Price (trips lead)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=lags,
    y=price_to_trips_corr,
    mode='lines+markers',
    name='Price → Trips (price leads)',
    line=dict(color='red', width=2)
))

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='Lagged Correlation: Does One Variable Predict the Other?',
    xaxis_title='Lag (months)',
    yaxis_title='Correlation Coefficient',
    height=500
)

fig.show()

print("\nGraph 4: If one line peaks at lag > 0, that variable predicts the other.")

### Graph 5: Vehicle Type & Age Breakdown

In [ ]:
# Vehicle type trends
top_types = price_by_type_pd.groupby('vehicle_type_clean')['listing_count'].sum().nlargest(6).index
price_by_type_filtered = price_by_type_pd[price_by_type_pd['vehicle_type_clean'].isin(top_types)]

fig = px.line(
    price_by_type_filtered,
    x='year_month',
    y='median_price',
    color='vehicle_type_clean',
    title='Median Car Price Over Time by Vehicle Type',
    labels={'median_price': 'Median Price ($)', 'year_month': 'Month'},
    height=500
)

fig.update_xaxes(tickangle=45)
fig.show()

# Vehicle age trends
fig = px.line(
    price_by_age_pd,
    x='year_month',
    y='median_price',
    color='age_category',
    title='Median Car Price Over Time by Vehicle Age',
    labels={'median_price': 'Median Price ($)', 'year_month': 'Month'},
    height=500
)

fig.update_xaxes(tickangle=45)
fig.show()

## 15. Analysis Complete

In [ ]:
print("\n" + "="*80)
print(f"  ANALYSIS COMPLETE ({' LAPTOP MODE' if LAPTOP_DEPLOYMENT else 'CLOUD MODE'})")
print("="*80)
print("\nNext steps:")
print("1. Review all graphs above")
print("2. Look for consistent patterns across multiple visualizations")
print("3. Consider alternative explanations for any correlations found")
print("4. Focus on pre-COVID data (2019) for cleaner signal")
print("5. Be aware of geographic mismatch (NYC taxi vs NY state cars)")
print("\n⚠️  Remember: Correlation ≠ Causation")
print("\n✓ Output files saved to data/ directory")